In [33]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd
from os import listdir
from os.path import isfile, isdir, join
from PIL import Image
import cv2
import math
#import argparse
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets('MNSIT_data',one_hot=True)

def rotate(image, angle, center=None, scale=1.0):
    # 獲取圖像尺寸
    (h, w) = image.shape[:2]
    # 若未指定旋轉中心，则將圖像中心設為旋轉中心
    if center is None:
        center = (w / 2, h / 2)
    # 執行旋轉
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    # 返回旋轉後的圖像
    return rotated

def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) #類似normal分布，產生隨機變量
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)
def conv2d(x,W):
    #stride [1,x_movement,y_movement,1],頭尾一定都是1
    #Must have strides[0]=strides[3]=1
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding = 'SAME') 
def max_pool_2x2(x):
    #stride [1,x_movement,y_movement,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
batch_size = 150
#batch_xs,batch_ys= mnist.train.next_batch(batch_size)
#print(batch_xs)
#print(batch_ys)

In [34]:
##將cvs裡面答案轉為陣列
df = pd.read_csv('validation.csv',header=None)  
df = df.T
df_temp = pd.get_dummies(df[0]) #One-hot編碼
y_r = np.array(df_temp)
##將驗證圖片讀檔，二值化並存成2維陣列200*(784)
img_set = np.zeros((200,28*28))

img_merge = np.zeros((20*28,10*28))
# 指定要列出所有檔案的目錄
mypath = "./validation"
# 取得所有檔案與子目錄名稱
files = listdir(mypath)
# 以迴圈處理
j=0
size = 28
for i,f in enumerate(files):
    # 產生檔案的絕對路徑
    fullpath = join(mypath, f)
    #print(f)
    # 判斷 fullpath 是檔案還是目錄
    ro = False
    if isfile(fullpath):
        im = cv2.imread(fullpath,0)
        image = cv2.imread(fullpath)
        img_merge[j%20*28:j%20*28+28,j//20*28: j//20*28+28]=im
        retval, im_bin = cv2.threshold(im,215, 255, cv2.THRESH_BINARY_INV)
        retval, labels,stats,centroids = cv2.connectedComponentsWithStats(im_bin)
        x,y,w,h,area=stats[1]
        cx,cy = centroids[1]

        if w<5:
            x=math.ceil(cx-3)
            w=w+4
        if h<5 :
            y=math.ceil(cy-3)
            h=h+6
        if w > h+1 and w >10 :
            #ro= True
            img = rotate(im,90)
            crop_img = img[x-1:x+w+1,y-1:y+h+1]
        else:
            crop_img = im[y:y+h,x:x+w]
        im_h ,im_w= crop_img.shape
        r = int(min(24/im_w,24/im_h))
        if r<=0: r=1
        if r>=4:r=2
        crop_img = cv2.resize(crop_img,(im_w*r,im_h*r),interpolation=cv2.INTER_AREA)#interpolation=cv2.INTER_NEAREST
        im_h ,im_w= crop_img.shape
        u = int((size-im_h)/2)
        v = int((size-im_w)/2)
        crop_img = cv2.copyMakeBorder(crop_img,u,size-im_h-u,v,size-im_w-v, cv2.BORDER_CONSTANT,value=[255,255,255])
        crop_img = cv2.erode(crop_img,(3,3), iterations=1)
        #cv2.imwrite('./Desktop/new/'+f,crop_img)
        
        img = np.array(crop_img,dtype=np.float32).reshape(28*28)
        img_set[j,:]=1-img/255
        j+=1

## 訓練

In [35]:
# define placeholder for inputs to network
xs=tf.placeholder(tf.float32,[None,784])/255. #28*28=784
ys=tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs,[-1,28,28,1]) #-1不論輸入資料維度，28*28資料像素，通道只有1是黑白
#print(x_image.shape) #[n_sample,28,28,1]

# Parameters
learning_rate = 5*1e-4
num_steps = 1500
batch_size = 150
display_step = 100
model_path = "my_project/save_net.ckpt"

#def network(x_image,learning_rate,num_steps,batch_size,display_step,model_path):
##conv1 layer##
W_conv1=weight_variable([5,5,1,32]) #patch 5*5,in size 厚度 1,out size 32
b_conv1=bias_variable([32])
#print(W_conv1)
#print(b_conv1)
hid_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1) #output size 28*28*32
hid_pool1 =max_pool_2x2(hid_conv1)  #output size 14*14*32

##conv2 layer##
W_conv2=weight_variable([5,5,32,64]) #patch 5*5,in size 厚度 32(前一層來),out size 64
b_conv2=bias_variable([64])
hid_conv2 = tf.nn.relu(conv2d(hid_pool1,W_conv2)+b_conv2) #output size 7*7*64
hid_pool2 =max_pool_2x2(hid_conv2)  #output size 7*7*64

##func1 layer##
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
#[n_samples,7,7,64]->>[n_samples,7*7*64]
hid_pool2_flat = tf.reshape(hid_pool2,[-1,7*7*64])
hid_fc1 = tf.nn.relu(tf.matmul(hid_pool2_flat,W_fc1)+b_fc1)
hid_fc1_drop = tf.nn.dropout(hid_fc1,keep_prob)

##func2 layer##
W_fc2 = weight_variable([1024,10]) #output 有10種
b_fc2 = bias_variable([10]) 
y_conv = tf.matmul(hid_fc1_drop, W_fc2) + b_fc2
prediction = tf.nn.softmax(y_conv)
#    return prediction


#prediction = network(x_image,learning_rate,num_steps,batch_size,display_step,model_path)
##the error between prediction and real data##
#tf.reduce_sum(ys*tf.log(prediction))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=ys, logits=prediction)) #loss
#tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy) 
#這樣的系統，需要AdamOptimizer來優化，且需較小的learning rate=0.0001

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(ys, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()
tStart = time.time()#計時開始
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_steps+1):
        batch_xs,batch_ys= mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys, keep_prob: 0.5})
        if i%display_step==0 or i==num_steps:
            train_accuracy = compute_accuracy(mnist.test.images,mnist.test.labels)
            print("step %d, training accuracy %g"%(i, train_accuracy))
    #print(batch_xs)        
    tEnd = time.time()#計時結束
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)
    
print("It cost %f sec" % (tEnd - tStart))#會自動做近位        
#print("test accuracy %g"%accuracy.eval(feed_dict={xs:batch_xs,ys:batch_ys, keep_prob: 1.0}))    

AttributeError: module 'tensorflow' has no attribute 'placeholder'

## 測試

In [8]:
model_path = "my_project/save_net.ckpt"

init = tf.global_variables_initializer()
with tf.Session(graph=tf.get_default_graph()) as sess:
    sess.run(init)
    saver = tf.train.import_meta_graph(model_path+'.meta')
    saver.restore(sess,tf.train.latest_checkpoint("my_project/"))
    prediction_result = sess.run(tf.argmax(prediction, 1),feed_dict={xs:img_set, keep_prob: 1})

    print("Testing Accuracy:",compute_accuracy(img_set,y_r))
    #for i,img in enumerate()  
    print(prediction_result)
    #將結果寫入csv
    #dataframe = pd.DataFrame(prediction_result)
    #dataframe.to_csv("pre.csv",index=False,header=False)

   

INFO:tensorflow:Restoring parameters from my_project/save_net.ckpt
Testing Accuracy: 0.67
[7 2 8 8 1 8 3 4 4 5 5 6 6 3 2 2 1 1 0 0 2 6 8 8 2 3 3 8 4 5 5 6 6 0 3 3 1
 2 4 3 0 0 1 1 7 2 3 6 6 5 5 9 4 3 9 9 8 8 8 0 3 6 3 6 7 9 3 2 2 8 8 8 8 4
 0 0 4 7 7 3 0 6 3 4 6 2 8 5 2 8 8 1 8 4 1 1 7 8 0 8 8 7 8 8 4 8 4 8 4 4 4
 8 0 2 1 6 8 1 1 8 7 4 3 0 3 8 7 6 3 2 2 3 3 8 5 6 6 4 9 8 8 8 8 8 8 8 8 8
 8 7 7 8 8 6 6 5 5 4 4 3 3 2 2 1 1 1 0 0 1 6 6 2 4 9 5 2 1 5 8 8 4 1 3 8 7
 7 3 3 5 3 1 3 6 2 3 7 2 3 3 7]
